In [1]:
using Distributed
nprocs()

1

In [3]:
addprocs(4)

4-element Array{Int64,1}:
  7
  8
  9
 10

In [4]:
nprocs()

10

In [8]:
@everywhere using Pkg
@everywhere Pkg.add("DrWatson")

      From worker 2:	   Updating registry at `~/.julia/registries/General`
      From worker 2:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Updating registry at `~/.julia/registries/General`


      From worker 2:	  Resolving package versions...
      From worker 6:	   Updating registry at `~/.julia/registries/General`

   Updating git-repo `https://github.com/JuliaRegistries/General.git`



      From worker 6:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 4:	   Updating registry at `~/.julia/registries/General`


  Resolving package versions...


      From worker 4:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 4:	  Resolving package versions...
      From worker 3:	   Updating registry at `~/.julia/registries/General`
      From worker 3:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 5:	   Updating registry at `~/.julia/registries/General`
      From worker 5:	   Updating git-repo `https://github.com/JuliaRegistries/General.git`
      From worker 5:	  Resolving package versions...
      From worker 2:	  Installed UnPack ─── v1.0.1
      From worker 2:	  Installed FileIO ─── v1.3.0
      From worker 6:	  Resolving package versions...
      From worker 4:	  Installed UnPack ─── v1.0.1
      From worker 4:	  Installed DrWatson ─ v1.14.3
      From worker 4:	  Installed FileIO ─── v1.3.0
      From worker 4:	   Updating `~/.julia/environments/v1.4/Project.toml`
      From worker 4:	  [634d3b9d] + DrWatson v1.14.3
      From worker 4:	   Updating `

   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
 [no changes]
   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
 [no changes]


      From worker 5:	   Updating `~/.julia/environments/v1.4/Project.toml`
      From worker 5:	  [634d3b9d] + DrWatson v1.14.3
      From worker 5:	   Updating `~/.julia/environments/v1.4/Manifest.toml`
      From worker 5:	  [634d3b9d] + DrWatson v1.14.3
      From worker 5:	  [5789e2e9] + FileIO v1.3.0
      From worker 5:	  [ae029012] + Requires v1.0.1
      From worker 5:	  [3a884ed6] + UnPack v1.0.1
      From worker 6:	  Installed FileIO ─── v1.3.0
      From worker 6:	   Updating `~/.julia/environments/v1.4/Project.toml`
      From worker 6:	  [634d3b9d] + DrWatson v1.14.3
      From worker 6:	   Updating `~/.julia/environments/v1.4/Manifest.toml`
      From worker 6:	  [634d3b9d] + DrWatson v1.14.3
      From worker 6:	  [5789e2e9] + FileIO v1.3.0
      From worker 6:	  [ae029012] + Requires v1.0.1
      From worker 6:	  [3a884ed6] + UnPack v1.0.1
      From worker 3:	  Installed FileIO ─── v1.3.0
      From worker 3:	   Updating `~/.julia/environments/v1.4/Project.toml`
     

In [35]:
@everywhere using Pkg
@everywhere Pkg.add("SharedArrays")

      From worker 6:	  Resolving package versions...


  Resolving package versions...


      From worker 4:	  Resolving package versions...
      From worker 2:	  Resolving package versions...


   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`


      From worker 5:	  Resolving package versions...
      From worker 3:	  Resolving package versions...


  [1a1011a3] + SharedArrays 
   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
 [no changes]


      From worker 6:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 4:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 6:	  [1a1011a3] + SharedArrays 
      From worker 6:	   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
      From worker 6:	 [no changes]
      From worker 4:	  [1a1011a3] + SharedArrays 
      From worker 4:	   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
      From worker 4:	 [no changes]
      From worker 2:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 3:	   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 2:	  [1a1011a3] + SharedArrays 
      From worker 2:	   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
      From worker 2:	 [no changes]
      From worker 5:	  [1a1011a3] + SharedArrays 
    

In [5]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
@everywhere using BenchmarkTools

@everywhere function benchmark_distributed_evaluation(parameters,N)
    errors = zeros(N) # init model errors
    # we need to read the experiment data in this block
    input_exp = Float64[]
    output_exp = Float64[]
    for row in CSV.File(datadir("experiment/new-data.csv"); delim = " ")
         push!(input_exp, row.I)
         push!(output_exp, row.O)
    end
    # done reading 
    
    F = Array{Future}(undef,N)
    
    for i in 1:N 
        F[i] = @spawnat i+1 (Catalyst.solve(parameters[1], parameters[2], 1.,
                                      input_exp, output_exp, progress=false, 
                                      microcomp_type=:nonlinear,
                                      Q=parameters[3], kₙ=parameters[4],
                                      calibration=true))
        end 
    
    for i in 1:N
    errors[i] = fetch(F[i])
    end
    return errors
end

parameters = [1e-4, 1., 1., 10.]
@time benchmark_distributed_evaluation(parameters,9)

┌ Info: Precompiling Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
└ @ Base loading.jl:1260


      From worker 2:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 6:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 7:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 8:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 4:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 9:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 3:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 10:	 Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`


┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
│   exception =
│    Cache file "/home/amjalled/.julia/compiled/v1.4/Catalyst/J9Gan_O9VyG.ji" not found.
│    
└ @ Base loading.jl:1041
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
│   exception =
│    Cache file "/home/amjalled/.julia/compiled/v1.4/Catalyst/J9Gan_O9VyG.ji" not found.
│    
└ @ Base loading.jl:1041
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
│   exception =
│    Cache file "/home/amjalled/.julia/compiled/v1.4/Catalyst/J9Gan_O9VyG.ji" not found.
│    
└ @ Base loading.jl:1041
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
│   exception =
│    Cache file "/home/amjal

1104.254044 seconds (21.58 M allocations: 1.035 GiB, 0.06% gc time)


9-element Array{Float64,1}:
 183.03812687209208
 183.03812687209208
 183.03812687209208
 183.03812687209208
 183.03812687209208
 183.03812687209208
 183.03812687209208
 183.03812687209208
 183.03812687209208